In [132]:
from openai import OpenAI
import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import matplotlib.pyplot as plt

In [133]:
ems = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']

In [134]:
def emotions_to_categorical(df):
    res = []

    for i in df['emotions']:
        tmp = [0 for _ in range(28)]
        for j in i:
            tmp[j] = 1
        res.append(tmp)
    tmp_df = pd.DataFrame(res, columns=ems)
    
    return tmp_df

In [135]:
def emotions_to_ekman(df):
    # anger disgust fear joy sadness surprise neutral
    ekman = [3, 3, 0, 0, 3, 3, 5, 5, 3, 4, 0, 1, 4, 3, 2, 3, 4, 3, 3, 2, 3, 3, 5, 3, 4, 4, 5, 6]
    res = []

    for i in df:
        tmp = [0, 0, 0, 0, 0, 0, 0]
        for j in range(len(i)):
            if i[j] == 1:
                tmp[ekman[j]] = 1
        res.append(tmp)
    tmp_df = pd.DataFrame(res, columns=['angry', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'])
    
    return tmp_df

In [136]:
def data_init(path = "../../data/dev.tsv", c = True):
    df = pd.read_csv(path, sep="\t", encoding = "utf-8", header=None)
    df.columns = ['text', 'emotions', 'id']
    df['emotions'] = list(map(lambda s : list(map(int, s.split(','))), df['emotions']))
    df = pd.concat([df, emotions_to_categorical(df)], axis=1)
    df = df.drop(columns=['emotions', 'id'])
    df['text'] = list(map(lambda s : s.replace('\\', '\\\\').replace('"', '\\"'), list(df['text']))) 
    if c:
        return df.iloc[:2500, :]
    else:
        return df

In [137]:
def evaluation(original_df, emotion_res):
    emotions_list = ems
    df = original_df
    predicted_df = pd.DataFrame(data = [[0 for _ in range(28)] for _ in range(len(df))], columns=emotions_list)
    for i in range(len(emotion_res)):
        for j in emotion_res[i]:
            if j in emotions_list:
                predicted_df.loc[i, j] = 1
    predicted = predicted_df.to_numpy()
    original = df.to_numpy()
    
    
    accuracy = accuracy_score(original, predicted)
    
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        original, predicted, average='micro'
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        original, predicted, average='macro'
    )
    
    precision_per_label, recall_per_label, f1_per_label, _ = precision_recall_fscore_support(
        original, predicted, average=None
    )

    precision_macro_std = np.std(precision_per_label)
    recall_macro_std = np.std(recall_per_label)
    f1_macro_std = np.std(f1_per_label)

    print("--- 모델 평가 결과 ---")
    print(f"전체 샘플에 대한 정확도 (Exact Match Accuracy): {accuracy:.4f}")
    print("\n--- Micro 평균 지표 ---")
    print(f"Precision (Micro): {precision_micro:.4f}")
    print(f"Recall (Micro): {recall_micro:.4f}")
    print(f"F1-Score (Micro): {f1_micro:.4f}")
    print("\n--- Macro 평균 지표 ---")
    print(f"Precision (Macro): {precision_macro:.4f}")
    print(f"Recall (Macro): {recall_macro:.4f}")
    print(f"F1-Score (Macro): {f1_macro:.4f}")
    
    print("\n--- 라벨별 지표 ---")
    for i in range(len(emotions_list)):
        print(f"{emotions_list[i]} - Precision: {precision_per_label[i]:.4f}, Recall: {recall_per_label[i]:.4f}, F1-Score: {f1_per_label[i]:.4f}")
    
    print(f"\nPrecision (Macro) 표준편차: {precision_macro_std:.4f}")
    print(f"Recall (Macro) 표준편차: {recall_macro_std:.4f}")
    print(f"F1-Score (Macro) 표준편차: {f1_macro_std:.4f}")

    return accuracy, f1_micro, f1_macro, precision_recall_fscore_support(original, predicted, average='macro')

In [138]:
def evaluation_ekman(original_df, emotion_res):
    emotions_list = 'anger disgust fear joy sadness surprise neutral'.split()
    predicted_df = pd.DataFrame(data = [[0 for _ in range(28)] for _ in range(len(original_df))], columns=ems)
    for i in range(len(emotion_res)):
        for j in emotion_res[i]:
            if j in ems:
                predicted_df.loc[i, j] = 1
    predicted = emotions_to_ekman(predicted_df.to_numpy()).to_numpy()
    original = emotions_to_ekman(original_df.to_numpy()).to_numpy()

    accuracy = accuracy_score(original, predicted)
    
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        original, predicted, average='micro'
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        original, predicted, average='macro'
    )
    
    precision_per_label, recall_per_label, f1_per_label, _ = precision_recall_fscore_support(
        original, predicted, average=None
    )

    precision_macro_std = np.std(precision_per_label)
    recall_macro_std = np.std(recall_per_label)
    f1_macro_std = np.std(f1_per_label)

    print("--- 모델 평가 결과 ---")
    print(f"전체 샘플에 대한 정확도 (Exact Match Accuracy): {accuracy:.4f}")
    print("\n--- Micro 평균 지표 ---")
    print(f"Precision (Micro): {precision_micro:.4f}")
    print(f"Recall (Micro): {recall_micro:.4f}")
    print(f"F1-Score (Micro): {f1_micro:.4f}")
    print("\n--- Macro 평균 지표 ---")
    print(f"Precision (Macro): {precision_macro:.4f}")
    print(f"Recall (Macro): {recall_macro:.4f}")
    print(f"F1-Score (Macro): {f1_macro:.4f}")
    
    print("\n--- 라벨별 지표 ---")
    for i in range(len(emotions_list)):
        print(f"{emotions_list[i]} - Precision: {precision_per_label[i]:.4f}, Recall: {recall_per_label[i]:.4f}, F1-Score: {f1_per_label[i]:.4f}")
    
    print(f"\nPrecision (Macro) 표준편차: {precision_macro_std:.4f}")
    print(f"Recall (Macro) 표준편차: {recall_macro_std:.4f}")
    print(f"F1-Score (Macro) 표준편차: {f1_macro_std:.4f}")

    return accuracy, f1_micro, f1_macro, precision_recall_fscore_support(original, predicted, average='macro')

In [191]:
def file_init():
    file_dict = {}
    basicPath = '../prompt/multi_v1'
    file_names = {
        'tier1_persona': basicPath+'/tier1/persona.txt',
        'tier1_guidelines': basicPath+'/tier1/guidelines.txt',
        'tier1_output_structure': basicPath+'/tier1/output_structure.txt',
        'tier1_fewshot': basicPath+'/tier1/fewshot.txt',
        'tier2_persona': basicPath+'/tier2/persona.txt',
        'tier2_guidelines': basicPath+'/tier2/guidelines.txt',
        'tier2_output_structure': basicPath+'/tier2/output_structure.txt',
        'tier2_fewshot': basicPath+'/tier2/fewshot.txt',
        'tier2_description': basicPath+'/tier2/emotion_description.txt',
        'tier3_persona': basicPath+'/tier3/persona.txt',
        'tier3_guidelines': basicPath+'/tier3/guidelines.txt',
        'tier3_output_structure': basicPath+'/tier3/output_structure.txt',
        'tier3_description': basicPath+'/tier3/emotion_description.txt',
        'tier3_fewshot': basicPath+'/tier3/fewshot.txt'
    }
    for key, value in file_names.items():
        file = open(value, 'r')
        file_dict[key] = file.read()
        file.close()
    return file_dict

In [192]:
data = data_init()

In [193]:
key_file = open('../key/openai_key.txt', 'r')
api_key = key_file.readline()
key_file.close()
client = OpenAI(api_key=api_key)

In [194]:
class Agent:
    def __init__(self, system, format, parent, child, filename, outputCol, input, inputCol):
        self.query = {
        "custom_id": "",
        "method": "POST",
        "url": "/v1/responses",
        "body": {
                "model": "gpt-4o-mini",
                "temperature": 0.5,
                "top_p": 1.0,
                "input": [{
                    "role": "developer",
                    "content": f"{system}"
                }, 
                {
                    "role": "user",
                    "content": ""
                }], 
                "max_output_tokens": 1000,
                "text": {
                    "format": format
                }
            }
        }
        self.parent = parent
        self.child = None
        if isinstance(child, list):
            self.child = child
        else:
            self.child = [child]
        self.filename = filename
        self.batch_id = None
        self.batch_res = None
        self.emotion_res = []
        self.outputCol = outputCol
        self.inputCol = inputCol
        self.input = input
        self.output = None
    
    def make_jsonl(self):
        tmp_query = self.query
        with open(self.filename, "w") as f:
            k = 0
            for i in range(len(self.input)):
                record = {}
                for j in self.inputCol:
                    record[j] = self.input.loc[i, j]
                tmp_query['custom_id'] = f"query{k}"
                tmp_query['body']['input'][1]['content'] = json.dumps(record)
                k += 1
                f.write(json.dumps(tmp_query) + "\n")
            
    def run(self):
        batch_input_file = client.files.create(
            file=open(self.filename, "rb"),
            purpose='batch'
        )
        print(batch_input_file)

        batch_input_file_id = batch_input_file.id
        create_batch=client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/responses",
            completion_window="24h",
        )
        self.batch_id = create_batch.id

    def check_status(self):
        batch = client.batches.retrieve(self.batch_id)
        result = None
        if batch.status == 'completed':
            out = batch.output_file_id
            if out != None:
                print('done!')
                result = client.files.content(out)
                self.batch_res = result
                return 1
            else:
                print('error')
                result = client.files.content(batch.error_file_id).text
                self.batch_res = result
                return -2
        elif batch.status == 'failed':
            print('failed')
            print(batch.errors)
            return -1
        else:
            print('it does not finish yet')
            print(batch.status)
            print(batch.request_counts)
            return 0
    
    def parse_result(self):
        status = self.check_status()
        if status != 1:
            print('can not parse result')
            return
        json_res = []
        for i in self.batch_res.text.split('\n')[:-1]:
            json_res.append(json.loads(i))

        for i in json_res:
            tmp = [[] for _ in range(len(self.outputCol))]
            n = json.loads(i['response']['body']['output'][0]['content'][0]['text'])

            for j in n['analysis']:
                for k in range(len(self.outputCol)):
                    tmp[k].append(j[self.outputCol[k]])

            self.emotion_res.append(tmp)
        tmp_df = pd.DataFrame(data=self.emotion_res, columns=self.outputCol)
        self.output = pd.concat([self.input, tmp_df], axis=1)


In [195]:
multi_v1_tier1_format = {
    "type": "json_schema",
    "name": "result",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {
            "analysis": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "polarity": {
                            "type": "string",
                            "enum": ["positive", "negative", "ambiguous", "neutral"]
                        },
                    },
                    "required": ["polarity"],
                    "additionalProperties": False
                }
            }
        },
        "required": ["analysis"],
        "additionalProperties": False
    }
}

In [208]:
files = file_init()
tier1_system = f"{files['tier1_persona']}{files['tier1_guidelines']}{files['tier1_output_structure']}{files['tier1_fewshot']}"
tier1 = Agent(tier1_system, multi_v1_tier1_format, None, None, "../inputs/multi/v1/tier1.jsonl", ['polarity'], data, ['text'])

In [198]:
tier1.make_jsonl()
tier1.run()

FileObject(id='file-FPjN8pKY2WpYJkmzn85hPs', bytes=14934108, created_at=1764657402, filename='tier1.jsonl', object='file', purpose='batch', status='processed', expires_at=1767249402, status_details=None)


In [209]:
tier1.batch_id = 'batch_6927e95d94948190811ce30abf8fd8a8' # """적용 전"""
tier1.batch_id = 'batch_692e88fb08d081908bff558e9cb873d4' # """1차 적용 후"""
print(tier1.batch_id)

batch_692e88fb08d081908bff558e9cb873d4


In [210]:
tier1.parse_result() 

done!


In [218]:
ne = ['neutral']
a = ['confusion', 'curiosity', 'realization', 'surprise']
n = ['anger', 'annoyance', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'fear', 'grief', 'nervousness', 'remorse', 'sadness']
p = ['admiration', 'amusement', 'approval', 'caring', 'desire', 'excitement', 'gratitude', 'joy', 'love', 'optimism', 'pride', 'relief']

In [219]:
res_p = [[] for _ in range(len(data))]
for i in ems:
    t = data[i].to_list()
    val = None
    if i in p:
        val = 'positive'
    elif i in n:
        val = 'negative'
    elif i in a:
        val = 'ambiguous'
    else:
        val = 'neutral'
    for j in range(len(data)):
        if t[j] == 1:
            res_p[j].append(val)

In [220]:
from sklearn.preprocessing import MultiLabelBinarizer
encoder = MultiLabelBinarizer()
encoder.fit([['positive', 'negative', 'ambiguous', 'neutral']])
true = encoder.transform(res_p)
pred = encoder.transform(tier1.output.polarity.to_list())

In [ ]:
"""
적용 전: (0.5750233703294786, 0.5521522732602462, 0.55042804336661, None) 0.532
1차: (0.5711651813770668, 0.5468898755775081, 0.547712272908647, None) 0.5324
"""
print(precision_recall_fscore_support(y_true=true, y_pred=pred, average='macro'))
print(accuracy_score(y_true=true, y_pred=pred))

(0.5711651813770668, 0.5468898755775081, 0.547712272908647, None)
0.5324


In [ ]:
# 적용 전
# positive 0.38333333333333336 0.4709897610921502 0.4226646248085758
# negative 0.5071090047393365 0.7109634551495017 0.5919778699861687
# ambiguous 0.5798045602605864 0.4517766497461929 0.5078459343794579
# neutral 0.8298465829846583 0.5748792270531401 0.6792237442922374

# 적용 후
# positive 0.3792134831460674 0.46075085324232085 0.41602465331278893
# negative 0.5073529411764706 0.6877076411960132 0.5839210155148096
# ambiguous 0.5727848101265823 0.4593908629441624 0.5098591549295775
# neutral 0.8253094910591472 0.5797101449275363 0.681044267877412
for i in range(4):
    print(['positive', 'negative', 'ambiguous', 'neutral'][i], precision_recall_fscore_support(y_true=true, y_pred=pred, average=None)[0][i], precision_recall_fscore_support(y_true=true, y_pred=pred, average=None)[1][i], precision_recall_fscore_support(y_true=true, y_pred=pred, average=None)[2][i])

positive 0.3792134831460674 0.46075085324232085 0.41602465331278893
negative 0.5073529411764706 0.6877076411960132 0.5839210155148096
ambiguous 0.5727848101265823 0.4593908629441624 0.5098591549295775
neutral 0.8253094910591472 0.5797101449275363 0.681044267877412


In [223]:
tier1.output.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,polarity
0,Is this in New Orleans?? I really feel like th...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,[ambiguous]
1,"You know the answer man, you are programmed to...",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,[negative]
2,I've never been this sad in my life!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,[negative]
3,The economy is heavily controlled and subsidiz...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,[neutral]
4,He could have easily taken a real camera from ...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,[neutral]


In [213]:
multi_v1_tier2_format = {
    "type": "json_schema",
    "name": "result",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {
            "analysis": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "ekman": {
                            "type": "string",
                            "enum": ["anger", "sadness", "joy", "neutral", "disgust", "surprise", "fear"]
                        },
                    },
                    "required": ["ekman"],
                    "additionalProperties": False
                }
            }
        },
        "required": ["analysis"],
        "additionalProperties": False
    }
}

In [214]:
files = file_init()
tier2_system = f"{files['tier2_persona']}{files['tier2_guidelines']}{files['tier2_output_structure']}{files['tier2_fewshot']}{files['tier2_description']}"
tier2 = Agent(tier2_system, multi_v1_tier2_format, tier1, None, "../inputs/multi/v1/tier2.jsonl", ['ekman'], tier1.output, ['text', 'polarity'])
tier1.child = [tier2]

In [215]:
tier2.make_jsonl()
tier2.run()

FileObject(id='file-UXoRBQfbNNvPbVnSk2YQAd', bytes=15816766, created_at=1764659445, filename='tier2.jsonl', object='file', purpose='batch', status='processed', expires_at=1767251445, status_details=None)


In [217]:
# tier2.batch_id = 'batch_6927ed8d4758819091542d2502fe2771' # 적용 전
# tier2.batch_id = 'batch_692e90f6bef48190908fa65c272ea9f6' # 1차 적용 후
print(tier2.batch_id)

batch_692e90f6bef48190908fa65c272ea9f6


In [163]:
tier2.parse_result()

done!


In [165]:
tier2.output.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,polarity,ekman
0,Is this in New Orleans?? I really feel like th...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,[ambiguous],[surprise]
1,"You know the answer man, you are programmed to...",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,[negative],[anger]
2,I've never been this sad in my life!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,[negative],[sadness]
3,The economy is heavily controlled and subsidiz...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,[neutral],[neutral]
4,He could have easily taken a real camera from ...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,[neutral],[neutral]


In [185]:
neutral = ['neutral']
joy = ['admiration', 'amusement', 'approval', 'caring', 'desire', 'excitement', 'gratitude', 'joy', 'love', 'optimism', 'pride', 'relief']
anger = ['anger', 'annoyance', 'disapproval']
fear = ['fear', 'nervousness']
surprise = ['confusion', 'curiosity', 'realization', 'surprise']
sadness = ['disappointment', 'embarrassment', 'grief', 'remorse', 'sadness']
disgust = ['disgust']

In [186]:
res_p = [[] for _ in range(len(data))]
for i in ems:
    t = data[i].to_list()
    val = None
    if i in joy:
        val = 'joy'
    elif i in sadness:
        val = 'sadness'
    elif i in surprise:
        val = 'surprise'
    elif i in anger:
        val = 'anger'
    elif i in fear:
        val = 'fear'
    elif i in disgust:
        val = 'disgust'
    else:
        val = 'neutral'
    for j in range(len(data)):
        if t[j] == 1:
            res_p[j].append(val)

In [187]:
from sklearn.preprocessing import MultiLabelBinarizer
encoder = MultiLabelBinarizer()
encoder.fit([['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']])
true = encoder.transform(res_p)
pred = encoder.transform(tier2.output.ekman.to_list())

In [ ]:
# 적용전 (0.4710720171373978, 0.48819752079183404, 0.46182573760718953, None) 0.49
print(precision_recall_fscore_support(y_true=true, y_pred=pred, average='macro'))
print(accuracy_score(y_true=true, y_pred=pred))

(0.4710720171373978, 0.48819752079183404, 0.46182573760718953, None)
0.49


In [189]:
t = precision_recall_fscore_support(y_true=true, y_pred=pred, average= None)

In [ ]:
# 적용 전
# anger 0.48606811145510836 0.4577259475218659 0.47147147147147145
# disgust 0.15286624203821655 0.47058823529411764 0.23076923076923078
# fear 0.47368421052631576 0.5094339622641509 0.4909090909090909
# joy 0.8489984591679507 0.5323671497584541 0.6543942992874109
# sadness 0.5483870967741935 0.5177664974619289 0.5326370757180157
# surprise 0.4017857142857143 0.46875 0.4326923076923077
# neutral 0.38571428571428573 0.46075085324232085 0.4199066874027994

for i in range(7):
    print(['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral'][i], t[0][i], t[1][i], t[2][i])

anger 0.48606811145510836 0.4577259475218659 0.47147147147147145
disgust 0.15286624203821655 0.47058823529411764 0.23076923076923078
fear 0.47368421052631576 0.5094339622641509 0.4909090909090909
joy 0.8489984591679507 0.5323671497584541 0.6543942992874109
sadness 0.5483870967741935 0.5177664974619289 0.5326370757180157
surprise 0.4017857142857143 0.46875 0.4326923076923077
neutral 0.38571428571428573 0.46075085324232085 0.4199066874027994


In [166]:
multi_v1_tier3_format = {
    "type": "json_schema",
    "name": "result",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {
            "analysis": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "emotion": {
                            "type": "string",
                            "enum": [ "admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion", "curiosity", "desire", "disappointment", "disapproval", "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief", "joy", "love", "nervousness", "optimism", "pride", "realization", "relief", "remorse", "sadness", "surprise", "neutral" ]
                        },
                        "reason": {
                            "type": "string"
                        }
                    },
                    "required": ["emotion", "reason"],
                    "additionalProperties": False
                }
            }
        },
        "required": ["analysis"],
        "additionalProperties": False
    }
}

In [167]:
files = file_init()
tier3_system = f"{files['tier3_persona']}{files['tier3_description']}{files['tier3_guidelines']}{files['tier3_output_structure']}{files['tier3_fewshot']}"
tier3 = Agent(tier3_system, multi_v1_tier3_format, None, None, "../inputs/multi/v1/tier3.jsonl", ['emotion', 'reason'], tier2.output, ['text', 'ekman'])


In [169]:
tier3.make_jsonl()
tier3.run()

FileObject(id='file-VaooBvTL9cisFR5N8FykAc', bytes=19178597, created_at=1764654649, filename='tier3.jsonl', object='file', purpose='batch', status='processed', expires_at=1767246649, status_details=None)


In [171]:
# tier3.batch_id = 'batch_6928179ae86881908fe301f3e3bf6bfb' """적용 전"""
# tier3.batch_id = 'batch_692e7e39d7d88190b31c870672677ba2' """적용 후"""
print(tier3.batch_id)

batch_692e7e39d7d88190b31c870672677ba2


In [179]:
tier3.parse_result()

done!


In [180]:
e = evaluation(data.iloc[:, 1:30], tier3.output['emotion'].to_list())
ek = evaluation_ekman(data.iloc[:, 1:30], tier3.output['emotion'].to_list())

--- 모델 평가 결과 ---
전체 샘플에 대한 정확도 (Exact Match Accuracy): 0.3076

--- Micro 평균 지표 ---
Precision (Micro): 0.3928
Recall (Micro): 0.3509
F1-Score (Micro): 0.3707

--- Macro 평균 지표 ---
Precision (Macro): 0.4309
Recall (Macro): 0.3174
F1-Score (Macro): 0.3108

--- 라벨별 지표 ---
admiration - Precision: 0.7377, Recall: 0.1974, F1-Score: 0.3114
amusement - Precision: 0.5133, Recall: 0.3919, F1-Score: 0.4444
anger - Precision: 0.2102, Recall: 0.7333, F1-Score: 0.3267
annoyance - Precision: 0.4000, Recall: 0.0286, F1-Score: 0.0533
approval - Precision: 0.1587, Recall: 0.0578, F1-Score: 0.0847
caring - Precision: 0.3220, Recall: 0.2923, F1-Score: 0.3065
confusion - Precision: 0.3333, Recall: 0.3088, F1-Score: 0.3206
curiosity - Precision: 0.3140, Recall: 0.2348, F1-Score: 0.2687
desire - Precision: 0.5000, Recall: 0.2647, F1-Score: 0.3462
disappointment - Precision: 0.2111, Recall: 0.2405, F1-Score: 0.2249
disapproval - Precision: 0.2308, Recall: 0.0432, F1-Score: 0.0727
disgust - Precision: 0.1701, Re

In [18]:
res = tier3.output.drop(columns=['reason'])

In [64]:
data1 = data_init()


In [65]:
res_li = [[] for _ in range(len(data1))]
for i in ems:
    t = data1[i].to_list()
    for j in range(len(data1)):
        if t[j] == 1:
            res_li[j].append(i)

In [84]:
dt = {}

In [86]:
tmp = res.emotion
txt = res.text
for i in range(2500):
    if 'disgust' in tmp[i]:
        if res_li[i] != tmp[i]:
            te = (tuple(res_li[i]), tuple(tmp[i]))
            if te in dt:
                dt[te].append(i)
            else:
                dt[te] = [i]
            print(i, res_li[i], tmp[i], txt[i], tier3.output.reason.iloc[i])

74 ['neutral'] ['disgust'] She's slept with so many guys she's starting to look like one ["The phrase 'starting to look like one' implies a strong negative judgment about the subject's behavior, indicating revulsion or strong disapproval."]
90 ['disgust', 'surprise'] ['disgust'] Wow, the level of evidence that you accept is more or less abysmal. ["The phrase 'abysmal' indicates a strong negative reaction and revulsion towards the level of evidence being discussed."]
148 ['neutral'] ['disgust'] He looks like his parents are blood related. ["The phrase 'blood related' implies a strong negative reaction to the idea of incest, suggesting revulsion or strong disapproval."]
204 ['admiration', 'disappointment'] ['disgust'] That's pretty gross. Unlucky OP. ["The phrase 'That's pretty gross' explicitly expresses a strong negative reaction to something unpleasant."]
297 ['disgust', 'embarrassment'] ['disgust'] That’s disgusting. You should be ashamed of yourself. Banging someone in sales. No exc

In [89]:
for i in [74, 148, 414, 707, 739, 748, 1192, 1193, 1249, 1267, 1274, 1325, 1389, 1417, 1604, 1690, 1754, 1755, 1782, 1986, 2000, 2122, 2410, 2475]:
    print(i, txt[i], tier3.output.reason.iloc[i])

74 She's slept with so many guys she's starting to look like one ["The phrase 'starting to look like one' implies a strong negative judgment about the subject's behavior, indicating revulsion or strong disapproval."]
148 He looks like his parents are blood related. ["The phrase 'blood related' implies a strong negative reaction to the idea of incest, suggesting revulsion or strong disapproval."]
414 Actually ugly women are not going to post their pics. ["The phrase 'actually ugly women are not going to post their pics' expresses a strong negative judgment and aversion towards the subject of unattractiveness."]
707 Yeah...when can the neckbeard asking this question feel okay if he masturbates? ["The phrase 'neckbeard asking this question' carries a derogatory connotation, indicating strong disapproval and revulsion towards the subject."]
739 From the looks of those fingers seems like you'll eat it anyway ["The phrase 'seems like you'll eat it anyway' implies a strong negative reaction t

In [87]:
for i in sorted(dt):
    print(i, dt[i])

(('admiration',), ('disgust',)) [512, 673]
(('admiration', 'approval', 'neutral'), ('disgust',)) [1900]
(('admiration', 'disappointment'), ('disgust',)) [204]
(('admiration', 'neutral'), ('disgust',)) [412]
(('amusement',), ('disgust',)) [1237, 2326, 2453]
(('anger',), ('disgust',)) [1747]
(('annoyance',), ('disgust',)) [1430]
(('annoyance', 'approval'), ('disgust',)) [957]
(('approval',), ('disgust',)) [1499, 1531, 1693]
(('approval', 'fear'), ('disgust',)) [1280]
(('approval', 'neutral'), ('disgust',)) [382]
(('caring', 'disgust'), ('disgust',)) [1702]
(('confusion',), ('disgust',)) [1574, 2201]
(('disappointment', 'disgust', 'fear'), ('disgust',)) [2279]
(('disappointment', 'embarrassment'), ('disgust',)) [559]
(('disapproval',), ('disgust',)) [1206, 1363, 1379, 1645, 1925, 2235]
(('disapproval', 'disgust'), ('disgust',)) [1565, 2391]
(('disapproval', 'neutral'), ('disgust',)) [599, 1913]
(('disgust',), ('disgust', 'amusement')) [843]
(('disgust', 'embarrassment'), ('disgust',)) [29

In [66]:
tup = list(map(tuple, list(map(set,res_li))))

In [67]:
d = {}

In [68]:
for i in tup:
    if i in d:
        d[i] += 1
    else:
        d[i] = 1

In [72]:
for i in sorted(d):
    print(i, d[i])

('admiration',) 152
('admiration', 'amusement') 3
('admiration', 'anger', 'love') 1
('admiration', 'annoyance') 1
('admiration', 'approval') 8
('admiration', 'caring') 1
('admiration', 'curiosity') 4
('admiration', 'disappointment') 3
('admiration', 'disappointment', 'amusement') 1
('admiration', 'disappointment', 'gratitude') 1
('admiration', 'excitement') 2
('admiration', 'gratitude') 11
('admiration', 'joy') 8
('admiration', 'love') 8
('admiration', 'love', 'approval') 1
('admiration', 'neutral') 4
('admiration', 'neutral', 'approval') 1
('admiration', 'optimism') 5
('admiration', 'optimism', 'caring') 1
('admiration', 'realization') 1
('amusement',) 97
('amusement', 'approval') 3
('amusement', 'caring') 1
('amusement', 'confusion') 1
('amusement', 'curiosity') 4
('amusement', 'excitement') 1
('amusement', 'joy') 4
('amusement', 'love') 1
('amusement', 'neutral') 8
('anger',) 56
('anger', 'amusement') 1
('anger', 'approval') 1
('anger', 'love') 2
('annoyance',) 79
('annoyance', 'amu